In [ ]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


In [ ]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [ ]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [ ]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [ ]:
print(df_train.shape,df_test.shape)

In [ ]:
df_train.head()

In [ ]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [ ]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [ ]:
valid_filenames

In [ ]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [ ]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [ ]:
print(df_train.shape,df_test.shape)

### Upsample minority classes

In [ ]:
df_train['genre_id'].value_counts()

In [ ]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [ ]:
df_train.tail()#,df_train.shape

In [ ]:
df_train['genre_id'].value_counts()

### Create Folds

In [ ]:
df_train['fold'] = 0

In [ ]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [ ]:
df_train.head()

In [ ]:
df_test.head()

#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
    [torchvision.transforms.CenterCrop((128,128))]
#     [torchvision.transforms.Resize((64,448))
     
#     ]
)

In [ ]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
                     tA.ShuffleChannels(p=0.05,mode="per_example",p_mode="per_example", sample_rate= 16000),
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.05,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [ ]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=True,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 2048,
                 N_FFT=512,
                 n_mels=64,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=True,
                 MakeThreeChannel= False
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.applyTukey = applyTukey
        self.quantizeData = quantizeData
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/{filename}')
        else:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/{filename}')
        return audio
    
    def apply_preprocess(self, waves):
        samples = np.hstack(waves)
        samples = waves / np.max(waves)
        return waves
        
    def quantize_data(self, data, classes):
        mu_x = self.mu_law_encoding(data, classes)
        return mu_x

    def mu_law_encoding(self,data, mu):
        mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)
        return mu_x

    def resample(self,audio,sr):
        
        audio = torchaudio.transforms.Resample(orig_freq= sr, new_freq=  16000, 
                                    resampling_method = 'sinc_interpolation', 
                                    lowpass_filter_width = 6, 
                                    rolloff = 0.99)(audio)
        
        
    
        return audio

    def makeCQT(self,audio,sr):
        audio = CQT1992v2(**CFG.qtransform_params,sr=sr,verbose=False)(audio)
        return audio
        
        
    def create_spectrogram(self,audio):
                
        specgram = torchaudio.transforms.MelSpectrogram(sample_rate=16000, 
                                                        n_fft=self.N_FFT, 
                                                        win_length=self.N_FFT, 
                                                        hop_length=self.HOP_LEN,
                                                        center=True,
                                                        pad_mode="reflect",
                                                        power=2.0,
                                                        norm='slaney',
                                                        onesided=True,
                                                        n_mels=self.n_mels,
                                                        mel_scale="htk"
                                                       )(audio)
        
        specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
#         specgram = specgram - specgram.min()
#         specgram = specgram/specgram.max()*255
        return specgram

    def __getitem__(self, index):
        row  = self.df.iloc[index]

        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        
        if samples.shape[0] != 2:
            samples = np.concatenate((samples,samples),0)
            assert samples.shape[0] == 2      
    
        samples = np.stack(samples, axis=0)
        
        if self.applyTukey:
            samples = samples*scipy.signal.tukey(samples.shape[-1],0.2)
        
        if self.MakeThreeChannel:
            mean = samples.mean(0).reshape(1,-1)
            samples = np.vstack((samples,mean))    

        samples = torch.from_numpy(samples).float().view(2,-1)
        

        if self.audiotfms is not None:
             samples = self.audiotfms(samples.unsqueeze(0)).squeeze(0)

#         samples = self.makeCQT(samples,sr=16000)
        
        samples = self.create_spectrogram(samples)
    
        if self.transform is not None:
            samples = self.transform(samples)

        if self.normalize:
            for i in range(samples.shape[0]):
                samples[i,:,:] = 255*(samples[i,:,:]-samples[i,:,:].min())/(samples[i,:,:].max()-samples[i,:,:].min())
            
        if self.quantizeData:
            samples = self.quantize_data(samples,19)
            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [ ]:
class POGFiles_v0(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 2048,
                 N_FFT=512,
                 n_mels=128,
                 transform=tfms):

        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            audio,sr = torchaudio.load(f'C:/Users/Kaggle/Pog_Music_Classification/train/{filename}')
        else:
            audio,sr = torchaudio.load(f'C:/Users/Kaggle/Pog_Music_Classification/test/{filename}')
        return audio,sr
    
    def create_spectrogram(self,audio,sr):
        specgram = torchaudio.transforms.MelSpectrogram(sample_rate=sr, 
                                                        n_fft=self.N_FFT, 
                                                        win_length=self.N_FFT, 
                                                        hop_length=self.HOP_LEN,
                                                        center=True,
                                                        pad_mode="reflect",
                                                        power=2.0,
                                                        norm='slaney',
                                                        onesided=True,
                                                        n_mels=self.n_mels,
                                                        mel_scale="htk"
                                                       )(audio)
        
        specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
        return specgram

        
    def __getitem__(self, index):
        row  = self.df.iloc[index]

        samples,sr = self.load_file(row.filename)
        if samples.shape[0] != 2:
            samples = np.concatenate((samples,samples),0)
            assert samples.shape[0] == 2      
        
        samples = np.stack(samples, axis=0)
        samples = torch.from_numpy(samples).float().view(2,-1)

        spec = self.create_spectrogram(samples,sr)
        
        if self.transform is not None:
            spec = self.transform(spec)
        
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return spec, label
        else:
            return spec

In [ ]:
a,b = next(iter(POGFiles_v0(df_train,transform=tfms)))
a.shape

In [ ]:
def gem_1d_res(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), 2,2).pow(1./p)

def gem_1d(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), (x.size(-1),)).pow(1./p )

In [ ]:
class ResBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, downsample=None):
        super(ResBlock, self).__init__()
        self.bn1 = nn.BatchNorm1d(num_features=in_channels)
        self.relu = nn.GELU()
        self.dropout = nn.Dropout(p=0.1, inplace=False)
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.downsample = downsample




    def forward(self, x):
        identity = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)

        if self.downsample is not None:
            out = gem_1d_res(out)
            identity = self.downsample(x)

        out += identity

        return out


class ECGNet(nn.Module):

    def __init__(self, struct=[14, 16, 18, 20], in_channels=2, fixed_kernel_size=17, num_classes=19):
        super(ECGNet, self).__init__()
        self.struct = struct
        self.planes = 16
        self.parallel_conv = nn.ModuleList()
        for i, kernel_size in enumerate(struct):
            sep_conv = nn.Conv1d(in_channels=in_channels, out_channels=self.planes, kernel_size=kernel_size,
                               stride=1, padding=0, bias=False)
            self.parallel_conv.append(sep_conv)


        self.bn1 = nn.BatchNorm1d(num_features=self.planes)
        self.relu = nn.ReLU(inplace=False)
        self.conv1 = nn.Conv1d(in_channels=self.planes, out_channels=self.planes, kernel_size=fixed_kernel_size,
                               stride=2, padding=2, bias=False)
        self.block = self._make_layer(kernel_size=fixed_kernel_size, stride=1, padding=8)
        self.bn2 = nn.BatchNorm1d(num_features=self.planes)
        self.rnn = nn.LSTM(input_size=2, hidden_size=40, num_layers=1, bidirectional=True)
        self.fc = nn.Linear(in_features=128, out_features=num_classes)        



    def _make_layer(self, kernel_size, stride, blocks=11, padding=0):
        layers = []
        downsample = None
        base_width = self.planes

        for i in range(blocks):
            if (i + 1) % 4 == 0:
                downsample = nn.Sequential(
                    nn.Conv1d(in_channels=self.planes, out_channels=self.planes + base_width, kernel_size=1,
                               stride=1, padding=0, bias=False),
                    nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
                )
                layers.append(ResBlock(in_channels=self.planes, out_channels=self.planes + base_width, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsample=downsample))
                self.planes += base_width
            elif (i + 1) % 2 == 0:
                downsample = nn.Sequential(
                    nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
                )
                layers.append(ResBlock(in_channels=self.planes, out_channels=self.planes, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsample=downsample))
            else:
                downsample = None
                layers.append(ResBlock(in_channels=self.planes, out_channels=self.planes, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsample=downsample))

        return nn.Sequential(*layers)



    def forward(self, x):
        out_sep = []

        for i in range(len(self.struct)):
            sep = self.parallel_conv[i](x)
            out_sep.append(sep)

        out = torch.cat(out_sep, dim=2)
        
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv1(out)  # out => [b, 16, 9960]

        out1 = self.block(out)
        out2 = self.block(out)
        out3 = torch.cat((out1,out2),-1)
        out = self.bn2(out3)
        out = self.relu(out)
        out = gem_1d(out)
        out = out.reshape(out.shape[0], -1)  # out => [b, 640]
        print(out.shape,x.shape)


        rnn_out, (rnn_h, rnn_c) = self.rnn(x.permute(2, 0, 1))
        new_rnn_h = rnn_h[-1, :, :]  # rnn_h => [b, 40]
        new_rnn_c = rnn_c[-1, :, :]
        new_out = torch.cat([out, new_rnn_h,new_rnn_c], dim=1)  # out => [b, 680]
        result = self.fc(new_out)  # out => [b, 20]
        return result


#### FastAI model

In [ ]:
df_train['genre_id'].nunique()

In [ ]:
df_train.head()

In [ ]:
import timm
timm.list_models()

In [ ]:
timm.create_model('efficientnet_b0')

In [ ]:
df_train['genre_id'].value_counts()

In [ ]:
df_train.head()

In [ ]:
# timm.create_model('tf_efficientnet_b2_ns')

In [ ]:
def run():
    oof = df_train.copy()
    oversampleTrain = True
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'efficientnet_b3_spec'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #FocalLoss()#LabelSmoothingCrossEntropy() ##CrossEntropyLossFlat() ##
        batch_size = 64
        n_epochs = 10

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,transform=tfms,quantizeData=False,normalize=False,
                            audiotfms=get_transforms(data='valid'))

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,transform=tfms,quantizeData=False,normalize=False,
                            audiotfms=get_transforms(data='valid'))

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        model = timm.create_model('efficientnet_b3', pretrained=False, in_chans=2)#get_model(resnet34)
        model.conv_stem.stride = (1,1)
        model.classifier = nn.Linear(1536,19)
#         model = EfficientNet_reduced_stride(backbone='efficientnet_b0')
#         model.fc = nn.Linear(2048,19)
#         model = ECGNet()
        f1_score = F1Score(average="micro")
        
        ### MixUp() in callbacks
#         opt_func=ranger,
        learn = Learner(dls, model,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True),
                                                           CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')])#.to_fp16()

        learn.fit_one_cycle(n_epochs, 1e-3, wd=1e-03)
#         learn.fit_flat_cos(n_epochs, 1e-3, wd=1e-03)

        learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')

        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,transform=tfms,quantizeData=False,normalize=False,
                           train=False,audiotfms=get_transforms(data='valid'))


        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [ ]:
df_test_valid.head()

In [ ]:
if __name__=='__main__':
    run()

In [ ]:
submission

### OOF

In [ ]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'efficientnet_b0_spec'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [ ]:
OOFOut.tail()

In [ ]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [ ]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [ ]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

In [ ]:
OOFOut[OOFOut['genre_id'].isin([17])][17].mean(),OOFOut[~OOFOut['genre_id'].isin([17])][17].mean()

In [ ]:
OOFOut[OOFOut['genre_id'].isin([18])][18].mean(),OOFOut[~OOFOut['genre_id'].isin([18])][18].mean()

In [ ]:
out = pd.DataFrame(np.zeros((5076,19)))
for fld in range(5):
    f = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fld}_probs.npy',allow_pickle=True)
    flat_list = pd.DataFrame([item.numpy() for sublist in f for item in sublist])
    out = out+flat_list
    
out/= 5

In [ ]:
# 022612 -> ID 1
# 024013 -> ID 0

In [ ]:
out.head()

In [ ]:
prob_ens_preds = np.argmax(np.array(out),1).reshape(-1,1)
df_sub = df_test_valid.copy()
df_sub['genre_id'] = prob_ens_preds

In [ ]:
df_sub.head()

In [ ]:
submission.head()

In [ ]:
out = submission.copy().drop('genre_id',1)
out = pd.merge(out,df_sub[['song_id','genre_id']],on='song_id',how='left')

In [ ]:
out.head()

In [ ]:
out.loc[out['song_id']==22612,'genre_id'] = 1
out.loc[out['song_id']==24013,'genre_id'] = 0

In [ ]:
out.head()

In [ ]:
out['genre_id'] = out['genre_id'].astype(int)
out.head()

In [ ]:
out.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_meanprobs.csv',index=False)

### Mode based sub

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### End ###